In [6]:
import torch

data = torch.load('fvmbv_sparse_keypoints.pth')
kpts_fixed = data['kpts_fixed']
kpts_moving = data['kpts_moving']
gt_displacements = data['gt_displacements']
del data

# Im Folgeden benötigte Variablen initialisieren.
# P = Points, D = Dimensions, kC = Amount of Candidates, E = Number of Edges for each Point
kC = 256
P,D = kpts_fixed.shape
E = 9

In [7]:
'''
Funktion die am ende benötigt wird um das Displacement zu berechnen.
Gegeben!
'''

def target_registration_error(my_disp_est,gt_displacements):
  diff = (my_disp_est*torch.Tensor([223/2,159/2,223/2]).view(-1,3)-gt_displacements*torch.Tensor([223/2,159/2,223/2]).view(-1,3))
  return torch.sqrt(diff.pow(2).sum(1)).mean()


'''
Funktion baut für die gegebene Keypoints eine Matrix mit den kC Kandidaten für jeden der kpts_fixed.
Rückgabe ist das relative displacement zwischen kpts_fixed und kpts_moving im Ausgangszustand.
'''
def find_candidates():
    # bauen M (kpt_fixed) x N (kpts_moving) matrix mit Pairwise Distance
    f_m_dist = torch.cdist(kpts_fixed, kpts_moving)
    # Index der jeweils 256 kandidaten für die 2048 bestimmen
    candidates_idx = torch.topk(input=f_m_dist,k=kC, dim=1,largest=False, sorted=True)[1]
    candidates_values = torch.gather(input=kpts_moving.view(-1,1,3).repeat(1,kC,1),dim=0,index=candidates_idx.view(-1,kC,1).repeat(1,1,D))
    return candidates_values - kpts_fixed.unsqueeze(1) # --> Realtives Displacment Berechnen

'''
Diese Methode baut aus den kpts_fixed einen Nachbarschaftsgraphen mit #E Nachbarn
Rückgabe ist hier eine Kantenliste
'''
def build_graph():
    # Hier alle Pairwise Distanzen bestimmen
    f_f_dis = torch.cdist(kpts_fixed,kpts_fixed)

    # Top 10 Werte nehmen und uns den 10ten anschauen als Schwellwert
    knn_value,_= torch.topk(input=f_f_dis,k=E+1,dim=1,largest=False, sorted=True)
    knn_highest_value = knn_value[:,E].unsqueeze(1)


    # Filtern der Distanzmatrix  pro Zeile nach Werten, die kleiner sind als der highest_value
    # IDEE_1 um auf 19020 zu kommen --> Threshold für die die Differenz
    # knn_mask = (f_f_dis < knn_highest_value+0.000896).int() - torch.eye(P,P)

    # IDEE_2 Gegenläufige Kanten mit rein nehmen. Angenommmen Kante 0 --> 1  aber keine Kante 1 --> 0,
    # da 1 andere 9 nächste Nachbarn haben kann. Dann nehmen wir die Kante 1 --> 0 trotzdem mit auf,
    # weil wir ja keinen gerichteten Graphen haben und die kante sogesehen eh existiert.
    knn_mask =  (f_f_dis < knn_highest_value).int()
    knn_mask_regular = knn_mask - torch.eye(P,P)
    knn_mask_tranposed = knn_mask.transpose(0,1) - torch.eye(P,P)
    knn_mask = knn_mask_regular + knn_mask_tranposed

    # mit non zero alle Indizes als Tupel speichern, bei denen eine Wert != 0 drin steht.
    return torch.nonzero(knn_mask)


### Aufgabe1 ###
candidates = find_candidates()
edges = build_graph()
print('Anzahl Kanten : ', edges.size()[0])
print('Anzahl Kandidaten pro Knoten (shape) : ', candidates.size()[0],'x',candidates.size()[1],'x',candidates.size()[2])

KeyboardInterrupt: 

In [3]:
### Aufgabe 2 ###
epochs = 7 # Wie viele Runden soll der Algorithmus laufen


# Speichert die Nachrichten pro Knoten
tmp_msg = torch.zeros(P, kC) # --> shape (2048x256)
# Nachrichten für jede Kante. Jede Nachricht hat 256 Kandidaten (0...255) an jeder Kante
msg = torch.zeros(len(edges), kC) # --> shape (#edgesx256)
# Speichert später die gesammelten Nachrichten aller Kanten
passed_message = torch.zeros(len(edges), kC) # --> shape (#edgesx256)


for epoch in range(epochs):
    # Betrachten hier jede Edge einzeln und berechnen den Message Term, da nicht parallelisiert.
    for edge_idx, edge in enumerate(edges):

        # Kante von p nach q
        p = edge[0]
        q = edge[1]

        # Zugehörige Kandidaten von p und q
        u_p = candidates[p]
        u_q = candidates[q]

        # Regularisierungsterm bauen als Punktweise Distanzen der Candidaten
        R = (u_p.unsqueeze(0) - u_q.unsqueeze(1)).pow(2).sum(2)

        # Berechnung der neuen Nachricht für jede Kante
        # Formel: Summe der Nachrichten bis zu Knoten P + 1.5 fachen Regularisierungsterm
        msg[edge_idx] = torch.min(passed_message[edge_idx].unsqueeze(0) + 1.5 * R, dim=1).values


    # Da wir jetzt alle Nachrichten entlang des Graphen berechnet haben, müssen wir nun unser tmp_msg updaten
    # Wir geben hier also die Nachrichten an die Nachbarknoten weiter.
    tmp_msg.zero_()
    tmp_msg.scatter_add_(dim=0,index=edges[:,1].view(-1,1).repeat(1,kC),src=msg)

    # Die Nachrichten in jedem Knoten werden hier noch durch den Graph gereicht, damit wir später
    # an jeder Kante durch zugriff auf den gleichen index in passed_message, die Summe erhalten.
    passed_message = torch.gather(input=tmp_msg,dim=0, index=edges[:,0].view(-1,1).repeat(1,kC))

    # berechnung des aktuellen Displaxements
    disp_pred = (torch.softmax(-50 * tmp_msg, dim=1).unsqueeze(-1) * candidates).sum(1)

    print('Epoche: ', epoch,'     Aktuelles Displacement: ', target_registration_error(disp_pred.squeeze(), gt_displacements).item(), 'mm')

Epoche:  0      Aktuelles Displacement:  13.523027420043945 mm
Epoche:  1      Aktuelles Displacement:  8.5609130859375 mm
Epoche:  2      Aktuelles Displacement:  1.2312085628509521 mm
Epoche:  3      Aktuelles Displacement:  0.8313611149787903 mm
Epoche:  4      Aktuelles Displacement:  0.6450588703155518 mm
Epoche:  5      Aktuelles Displacement:  0.502339243888855 mm
Epoche:  6      Aktuelles Displacement:  0.4164634943008423 mm
